# Registros de empleados

Se creara una aplicacion grafica con Python que gestione una base de datos de "empleados", entendiendose por gestion la Conexión a la base de datos, Creación de la tabla de empleados,Inserción, Consulta, actualización y Eliminación de un empleado y su informacion dentro de la empresa.

In [ ]:
### Importacion de librerias
import numpy as np 
import sqlite3 as sl
import tkinter as tk

## Conexion

## Creacion de tabla

## Insercion de empleados

## Consulta de informacion

## Actualizacion de informacion

## Eliminacion de informacion